In [1]:
import import_ipynb
import numpy as np
import sys
import csv
import numpy.ma as ma
import matplotlib.pyplot as plt
import a2_extract_list_meetings as a2
import c1_extract_paragraphe_issue as c1
import c2_extract_sentence_issue as c2
import e1_extract_occ_gen as e1
import e2_extract_occ_original as e2 
import g1_generate_dictionary as g1



importing Jupyter notebook from a2_extract_list_meetings.ipynb
importing Jupyter notebook from a1_compute_list_meeting.ipynb
importing Jupyter notebook from c1_extract_paragraphe_issue.ipynb
Issue  34
importing Jupyter notebook from c2_extract_sentence_issue.ipynb
importing Jupyter notebook from e1_extract_occ_gen.ipynb
importing Jupyter notebook from e2_extract_occ_original.ipynb
importing Jupyter notebook from g1_generate_dictionary.ipynb


In [2]:
def open_file(file):
    list_file = []
    f = open(file)
    for x in csv.reader(f):
        list_file.append(x)
    return list_file[1:]

In [3]:
def write_occurrence_issue(occurences_meetings, s):
        with open(s, "w", newline='') as file:
                writer = csv.writer(file)
        #header
                writer.writerow(('entity','interventions'))
                writer.writerows(occurences_meetings)

In [4]:
def combine_gen(dictionary, original_occ, generated_occ):
    combined = []

    for eo in original_occ:
        list_eo = []
        for eg in generated_occ:
            if(eg[0] in dictionary[eo[0]]):
                list_eo.append(eg)
        combined.append(list_eo)
    return combined

In [5]:
def create_stat_egals(generated_occ, original_occ,number):
    original = []
    generated = []
    for i in range(len(original_occ)):
        original.append((number,original_occ[i][0],original_occ[i][1]))
        g = sorted(generated_occ[i],key=lambda x: x[1], reverse=True)[0]
        generated.append((number,g[0],g[1]))
    return original,generated 

In [16]:
def extract_interventions(number, list_sentences):
    """Extract the intervention for both dataset and keep only the one that match the dictionary"""
    #Load the occurences for the original and generated file, and the dictionary
    original_occ = e2.extract_occurences_issue(number)
    gen_occ = e1.extract_occurences_issue_ENB(list_sentences, number)

    #dictionary = g1.compute_dictionary()

    #Combine element same key
    #generated_occ= combine_gen(dictionary, original_occ, gen_occ)
    # Original and generated occurence with only the entities into the dictionary
    #original,generated = create_stat_egals(generated_occ, original_occ, number)

    return original_occ,gen_occ

In [21]:
def extract__all_intervention():
    """Extract the intervention for all the issues in the original dataset and write them into .csv file"""
    # Extract all the issue number in the original dataset
    list_meetings = open_file('list_meetings.csv')
    list_issues = a2.issues_list(list_meetings)

    list_generated = []
    list_original = []

    for i in list_issues:
        if(45 < int(i) and int(i) < 594):
            list_paragraphes = c1.extract_paragraphes_from_issue(int(i))
            list_sentences = c2.sentences(list_paragraphes)
            original,generated = extract_interventions(int(i), list_sentences)

            list_generated += generated
            list_original += original
    return list_generated, list_original

    

In [22]:
list_generated, list_original = extract__all_intervention()

Issue  46
Issue  47
Issue  48
Issue  49
Issue  50
Issue  51
Issue  52
Issue  53
Issue  54
Issue  55
Issue  56
Issue  57
Issue  58
Issue  59
Issue  60
Issue  61
Issue  62
Issue  63
Issue  64
Issue  65
Issue  66
Issue  67
Issue  68
Issue  69
Issue  70
Issue  71
Issue  72
Issue  73
Issue  74
Issue  75
Issue  76
Issue  77
Issue  78
Issue  79
Issue  80
Issue  81
Issue  82
Issue  83
Issue  84
Issue  85
Issue  86
Issue  87
Issue  88
Issue  89
Issue  90
Issue  91
Issue  92
Issue  93
Issue  94
Issue  95
Issue  96
Issue  97
Issue  98
Issue  99
Issue  100
Issue  101
Issue  102
Issue  103
Issue  104
Issue  105
Issue  106
Issue  107
Issue  108
Issue  109
Issue  110
Issue  111
Issue  112
Issue  113
Issue  114
Issue  115
Issue  116
Issue  117
Issue  118
Issue  119
Issue  120
Issue  121
Issue  122
Issue  123
Issue  124
Issue  125
Issue  126
Issue  127
Issue  128
Issue  129
Issue  130
Issue  131
Issue  132
Issue  133
Issue  134
Issue  135
Issue  136
Issue  137
Issue  138
Issue  139
Issue  140
Issue  14

In [38]:
num_o = np.sum([int(x[1]) for x in list_original])
num_g = np.sum([int(x[2]) for x in list_generated])

In [39]:
num_o

24154

In [40]:
num_g

29853

d = define_dict_ref()

swiss, us, china, saudi_arabia= extract_occurence_by_list_ref(d)





import pandas as pd

index = np.arange(len(us))
df = pd.DataFrame({'Switzerland': swiss,'China': china, 'US': us, 'Saudi Arabia': saudi_arabia}, index=index)

ax = df.plot(figsize=(25,9),kind='bar',title='Number of intervention for each meetings between 1995-2013')
ax.set_xlabel("Meetings in chronological order")
ax.set_ylabel("Number of interventions")
